In [1]:
import pandas as pd
from pypinyin import pinyin, Style
import re

In [2]:
df = pd.read_csv('mean_result.csv')

df = df[['0', '5']]

def add_tones(transcription, pinyin_transcription):
    words = transcription.split(',')
    pinyin_words = pinyin_transcription.split(',')
    
    syllable_pattern = re.compile(r'[^\W\d_]+|\d+')
    
    updated_words = []
    
    for m in range(len(words)):
        cyrillic_chars = list(words[m])
        
        pinyin_syllables = pinyin_words[m].split()
        
        updated_syllables = []
        
        syllable_matches = syllable_pattern.findall(words[m])
        
        for k in range(len(syllable_matches)):
            #Получаем начальный и конечный индексы совпадения слогов
            start = words[m].find(syllable_matches[k])
            end = start + len(syllable_matches[k])
            
            #Берем тон из слога в пиньине
            tone = pinyin_syllables[k][-1]
            
            #Добавляем тон в конец слога на кириллице
            updated_char = cyrillic_chars[end - 1] + tone
            
            updated_syllable = words[m][start:end-1] + updated_char
            
            updated_syllables.append(updated_syllable)
        
        updated_transcription = ''.join(updated_syllables)
        
        updated_words.append(updated_transcription)
    
    return ''.join(updated_words)

#Создаем колонку с пиньинем
df['pinyin'] = df['0'].apply(lambda x: ' '.join([''.join(syllable) for syllable in pinyin(x, style=Style.TONE3)]))

df['5'] = df.apply(lambda x: add_tones(x['5'], x['pinyin']), axis=1)

df = df.drop('pinyin', axis=1)

df.to_csv('dataset_with_tones.csv', index=False)